In [26]:
import pandas as pd
import numpy as np
import math
from IPython.display import Markdown, display
from scipy.stats import skew
import seaborn as sns
import matplotlib.pyplot as plt    
from dataHack import removeColl 
%matplotlib inline
import warnings
import scipy
warnings.filterwarnings('ignore')
import networkx as nx
%load_ext autotime
def printmd(string):
    display(Markdown(string))
import plotly.offline as py1
py1.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
import difflib
import plotly
from nltk.metrics import jaccard_distance
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import plotly.plotly as py
from plotly.tools import FigureFactory as ff
plotly.tools.set_credentials_file(username='sushant.bharti', api_key='WptZURbEo6AnBZpSHNgO')
plt.style.use('seaborn-bright')
pal = sns.color_palette()

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


time: 756 ms


In [27]:
df_train = pd.read_csv("train.csv",nrows=5).fillna("")
df_test = pd.read_csv("test.csv").fillna("")

time: 11.2 s


In [28]:
stops = set(stopwords.words("english"))

time: 80 ms


In [29]:
def build_dict(sentences):
#    from collections import OrderedDict

    '''
    Build dictionary of train words
    Outputs: 
     - Dictionary of word --> word index
     - Dictionary of word --> word count freq
    '''
    print('Building dictionary..'),
    wordcount = dict()
    #For each worn in each sentence, cummulate frequency
    for ss in sentences:
        for w in ss:
            if w not in wordcount:
                wordcount[w] = 1
            else:
                wordcount[w] += 1
    
    worddict = dict()
    for idx, w in enumerate(sorted(wordcount.items(), key = lambda x: x[1], reverse=True)):
        worddict[w[0]] = idx+2  # leave 0 and 1 (UNK)

    return worddict, wordcount

time: 34 ms


In [30]:
def generate_sequence(sentences, dictionary):
    '''
    Convert tokenized text in sequences of integers
    '''
    seqs = [None] * len(sentences)
    for idx, ss in enumerate(sentences):
        seqs[idx] = [dictionary[w] if w in dictionary else 1 for w in ss]

    return seqs

time: 68 ms


In [31]:
 
def normalize(x):
    return x.lower().split()

time: 84 ms


In [32]:
questions = df_train['question1'].tolist() + df_train['question2'].tolist()

time: 66 ms


In [33]:
%reload_ext autotime
tok_questions = [normalize(s) for s in questions]
worddict, wordcount = build_dict(tok_questions)

Building dictionary..


In [34]:
print(np.sum(list(wordcount.values())), ' total words ', len(worddict), ' unique words')

(111, ' total words ', 72, ' unique words')
time: 92 ms


In [35]:
def jc(x):
    return jaccard_distance(set(x['s_question1']),set(x['s_question2']))
def cosine_d(x):
    a = set(x['s_question1'])
    b = set(x['s_question2'])
    d = len(a)*len(b)
    if (d == 0):
        return 0
    else: 
        return len(a.intersection(b))/d

time: 72 ms


In [36]:
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 1))
tfidf.fit_transform(questions)

<10x43 sparse matrix of type '<type 'numpy.float64'>'
	with 56 stored elements in Compressed Sparse Row format>

time: 82 ms


In [37]:
def diff_ratios(st1, st2):
    seq = difflib.SequenceMatcher()
    seq.set_seqs(str(st1).lower(), str(st2).lower())
    return seq.real_quick_ratio()

def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

time: 56 ms


In [38]:
def get_features(df_features):
    
    print('jaccard...')
    df_features['s_question1'] = generate_sequence(df_features['question1'].apply(normalize),worddict)
    df_features['s_question2'] = generate_sequence(df_features['question2'].apply(normalize),worddict)
    df_features['z_jaccard'] = df_features.apply(jc,axis = 1)
    
    print('cosine....')
    df_features['z_cosine'] = df_features.apply(cosine_d,axis = 1)
    
    print('length....')
    df_features['z_len1'] = df_features.question1.map(lambda x: len(str(x)))
    df_features['z_len2'] = df_features.question2.map(lambda x: len(str(x)))
    df_features['z_word_len1'] = df_features.question1.map(lambda x: len(str(x).split()))
    df_features['z_word_len2'] = df_features.question2.map(lambda x: len(str(x).split())) 
    
    print('difflib...')
    df_features['z_match_ratio'] = df_features.apply(lambda r: diff_ratios(r.question1, r.question2), axis=1)  #takes long
    
    print('word match...')
    df_features['z_word_match'] = df_features.apply(word_match_share, axis=1, raw=True)
    
    print('tfidf...')
    question1_tfidf = tfidf.transform(df_features.question1.tolist())
    question2_tfidf = tfidf.transform(df_features.question2.tolist())
    df_features['z_tfidf_sum1'] = np.sum(question1_tfidf, axis = 1)
    df_features['z_tfidf_sum2'] = np.sum(question2_tfidf, axis = 1)
    df_features['z_tfidf_mean1'] = np.mean(question1_tfidf, axis = 1)
    df_features['z_tfidf_mean2'] = np.mean(question2_tfidf, axis = 1)
    df_features['z_tfidf_len1'] = (question1_tfidf != 0).sum(axis = 1)
    df_features['z_tfidf_len2'] = (question2_tfidf != 0).sum(axis = 1)
    
    return df_features.fillna(0.0)

time: 63 ms


In [39]:
df_train = get_features(df_train)

jaccard...
cosine....
length....
difflib...
word match...
tfidf...
time: 100 ms


In [40]:
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate,s_question1,s_question2,z_jaccard,z_cosine,...,z_word_len1,z_word_len2,z_match_ratio,z_word_match,z_tfidf_sum1,z_tfidf_sum2,z_tfidf_mean1,z_tfidf_mean2,z_tfidf_len1,z_tfidf_len2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[5, 4, 2, 6, 7, 6, 21, 14, 18, 3, 11, 32, 3, 63]","[5, 4, 2, 6, 7, 6, 21, 14, 18, 3, 11, 67]",0.230769,0,...,14,12,0.926829,0,2.342687,2.121320,0.054481,0.049333,6,5
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[5, 4, 2, 53, 20, 19, 15, 71]","[5, 13, 31, 33, 2, 48, 36, 23, 2, 19, 15, 47, 62]",0.750000,0,...,8,13,0.733813,0,2.230896,2.819188,0.051881,0.065563,5,8
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[8, 10, 9, 40, 2, 12, 20, 72, 17, 68, 38, 58, ...","[8, 10, 17, 12, 25, 26, 7, 70, 29, 64]",0.800000,0,...,14,10,0.893939,0,2.442741,2.229275,0.056808,0.051844,6,5
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[39, 51, 9, 60, 50, 52, 8, 10, 9, 69, 55]","[41, 2, 59, 37, 65, 4, 35, 7, 28]",1.000000,0,...,11,9,0.869565,0,1.732051,2.138090,0.040280,0.049723,3,5
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[16, 27, 66, 3, 54, 44, 56, 46, 73, 22, 30, 49...","[16, 45, 13, 57, 3, 43, 24]",0.888889,0,...,13,7,0.678261,0,2.993785,1.993467,0.069623,0.046360,9,4


time: 89 ms


In [41]:
df_train.iloc[:,6:]

,s_question1,s_question2,z_jaccard,z_cosine,z_len1,z_len2,z_word_len1,z_word_len2,z_match_ratio,z_word_match,z_tfidf_sum1,z_tfidf_sum2,z_tfidf_mean1,z_tfidf_mean2,z_tfidf_len1,z_tfidf_len2
0,"[5, 4, 2, 6, 7, 6, 21, 14, 18, 3, 11, 32, 3, 63]","[5, 4, 2, 6, 7, 6, 21, 14, 18, 3, 11, 67]",0.230769,0,66,57,14,12,0.926829,0,2.342687,2.121320,0.054481,0.049333,6,5
1,"[5, 4, 2, 53, 20, 19, 15, 71]","[5, 13, 31, 33, 2, 48, 36, 23, 2, 19, 15, 47, 62]",0.750000,0,51,88,8,13,0.733813,0,2.230896,2.819188,0.051881,0.065563,5,8
2,"[8, 10, 9, 40, 2, 12, 20, 72, 17, 68, 38, 58, ...","[8, 10, 17, 12, 25, 26, 7, 70, 29, 64]",0.800000,0,73,59,14,10,0.893939,0,2.442741,2.229275,0.056808,0.051844,6,5
3,"[39, 51, 9, 60, 50, 52, 8, 10, 9, 69, 55]","[41, 2, 59, 37, 65, 4, 35, 7, 28]",1.000000,0,50,65,11,9,0.869565,0,1.732051,2.138090,0.040280,0.049723,3,5
4,"[16, 27, 66, 3, 54, 44, 56, 46, 73, 22, 30, 49...","[16, 45, 13, 57, 3, 43, 24]",0.888889,0,76,39,13,7,0.678261,0,2.993785,1.993467,0.069623,0.046360,9,4


time: 113 ms


In [42]:
df_test = get_features(df_test)

jaccard...
cosine....
length....
difflib...
word match...
tfidf...
time: 10min 26s


In [43]:
df_test.to_csv('df_test.csv')

time: 32.3 s


In [44]:
.

SyntaxError: invalid syntax (<ipython-input-44-8cf8463b34ca>, line 1)

In [ ]:
#generate_sequence(df_train['question2'].apply(normalize),worddict)

In [ ]:
#del df_features

In [ ]:
df_features

In [ ]:
jc(df_features.iloc[1,:2]),df_features.iloc[1,:2]

In [ ]:
df_features.question1 =df_train.question1

In [ ]:
df_features.question1

In [ ]:
x=tfidf.transform(df_features.question1.tolist())

In [ ]:
tfidf.idf_

In [ ]:
 np.sum(x, axis = 1)

In [ ]:
 np.sum(x, axis = 0)